In [4]:
from bs4 import BeautifulSoup # For HTML parsing
import requests# Website connections
import re # Regular expressions
from time import sleep # To prevent overwhelming the server between connections
from collections import Counter # Keep track of our term counts
from nltk.corpus import stopwords # Filter out stopwords, such as 'the', 'or', 'and'
import pandas as pd # For converting results to a dataframe and bar chart plots
%matplotlib inline

In [73]:
url = 'https://www.indeed.com/jobs?q=data+scientist&l=New+York%2C+NY'
req = requests.get(url)
source = req.text  

bs_tree = BeautifulSoup(source, 'html.parser')
print(bs_tree.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en">
 <head>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <script src="/s/798993d/en_US.js" type="text/javascript">
  </script>
  <script>
   window.indeed=window.indeed||{};indeed.crashtext=indeed.crashtext||{};indeed.crashtext.populate=indeed.crashtext.populate||function(c,g){var d=window[g];if(d){var e=d[""]||{},h=e.salt;if(e.hasOwnProperty("salt"))for(var b in c){for(var a=void 0,a=h,f=b.length;f;)a=33*a^b.charCodeAt(--f);a>>>=0;e.hasOwnProperty("id_length")&&(a=String(a).substring(0,e.id_length));d[a]=c[b]}else for(b in c)d[b]=[null].concat(c[b])}};indeed.crashtext.populate({"indeedapply_serp_label":["Express apply"],"Turn on":["Subscribe"],"See new jobs for this search":["Get email updates on this search"]}, 'indeed.i18n.localeData')
  </script>
  <link href="/s/114f565/jobsearch_all.css" rel="stylesheet" type="text/css"/>
  <link href="http://rss.indeed.com/rss?q=data+scientist&amp;l=New+York%2C+NY" rel="alternate"

In [18]:
job_page_string = bs_tree.find(id = 'searchCountPages').contents[0]
job_page_string = job_page_string.split()[-2]
job_page_string

'1,048'

In [24]:
page_count_digits = [int(i) for i in job_page_string if i.isdigit() ]
page_count_digits

[1, 0, 4, 8]

In [30]:
num_pages = 0
exp = len(page_count_digits) -1
for i in page_count_digits:
    num_pages = num_pages + i*(10**exp)
    exp -= 1
print(num_pages)

1048


In [90]:
from urllib.parse import urljoin
import time
base_url = 'http://www.indeed.com'
job_links = []

for i in range(1,5):    #搜索结果1-3页
    
    # 为第i页创建 bs object
    url = 'https://www.indeed.com/jobs?q=data+scientist&l=New+York%2C+NY&start=' + str(i*10)    #创建第i页的url   
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
    
    divs = soup.find_all('div', class_="title")    # each link is under a div tab
    href_links = [item.a.get('href') for item in divs]    # create a list of all href links
    
    
    for href_link in href_links:
        job_links.append(urljoin(base_url, href_link))    # create招聘页面的完整网址，并将其添加到 job_links 列表中
    
    time.sleep(0.5)    # 怕把服务器累死，be nice

In [91]:
len(job_links)

68

In [92]:
for i in job_links:
    print(i)

http://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0AbcLYJH6EEU2hhfUhIe4V-wtZUPXOEfJh72XLwlGjJwXW5GoFa_FpED8Nwq_yXdXN4COdSxaYIyezRVKWS7LHiErenmbowZSGENzUVg1n4jUZD7gC_eG4ul3vBbTTZAepVH-W7Pyx_ISf5K5j_0ykq9uk81DCs2MqIDezByJ5BB34kEW37A_y5xuV-hql8lP2AVfjUDEHdmCBF_eSXcO-5B75_gFkLnrCiPA7ffviqgxk7vp0A3h3qvPxGjOjr5K4CiW6mCmdIcBrXDZwm8W4n_EyAheOx8ASq4Fl8os3lh6zvH4ZcH91N2SZM7rYJpM_jgUxRavL5kJ3ZQSIgPDy3Q5VRs1LDhxgNGkOI2sP9wcu9OCBl70pRFOEtiFmEQnmPqK5I3bZCaiDjAp8FCijUYPYZJnAl4BA=&p=0&fvj=0&vjs=3
http://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BJV66PMW1vfRBroK9Xo9q-fe_BYhfQZJW05MGq2kEzWw4JaEZszoVltZBXHzrcp7-fE73SeB1zSIxDV3cxq3Jov-dSviG7u-bqRA-0oWJU0yhdCI-xbmL5l_6Wq94PKELgDMJ2ZAtWpgUoAfyOaUQ81StWKjf3c2AJ9jq7ocHQ8gPBlvMyQ-t6lc4xycuEwC9RqFs2u3afa8VEFSsGDCd9thQZc2LqtEDRLPT6RUR39TN6bJ6YTt9ItcQsaYgB_M3XQFbo09-26gBGcLG-GIeh6kfqo5i14Cj95ga_OLN_a_GKa1e4kCitTb6cbEdD7UTBsJ9PKXi7PcoZ8XAVNNkAhW2mgUbAkWtTBP0NPIswv_wZ1x5l-5RX8CeAIY36NA1j0YhPLIZY9D2Jp8mKETapCO-tNwEy_YMz_p72UY9xLOSz8il5BIW6hKNiVhiatUYmmjT327N9b3lL0